# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

**Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. Há linhas duplicadas?**

In [1]:
import pandas as pd
import requests
import numpy as np

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

**Conte o número de valores missing por variável.**


In [2]:
# 2) seu código aqui

sinasc.isna()
sinasc.isnull().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

**Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam.**

**Refaça a contagem de valores missings.**




In [3]:
# 3) seu código aqui
variaveis = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
df = pd.DataFrame(sinasc)
df = df[variaveis]
df.isnull().sum()


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

**Apgar é uma nota que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de missings.**

In [4]:
df['APGAR5'].shape


(27028,)

In [5]:
# 4) seu código aqui
df_APGAR5 = df['APGAR5'].dropna(axis = 0 , how = 'any')
print(f"NA_Values: {df['APGAR5'].isnull().sum()}")

print(f'\nNova quantia de NA_Values: {df_APGAR5.isnull().sum()}')
print(f"Linhas: {df_APGAR5.count()}")


sinasc['APGAR5'] = df_APGAR5

NA_Values: 103

Nova quantia de NA_Values: 0
Linhas: 26925


**observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. Vamos assumir que o não preenchido é o mesmo que o código 9.**

In [6]:
# 5) seu código aqui
var_cod_9 =  ['ESTCIVMAE', 'CONSULTAS'] 
df_cod_9 = pd.DataFrame(sinasc)
df_cod_9 = df[var_cod_9]

df_cod_9 = df_cod_9.replace(9, np.NaN)

df_cod_9 = df_cod_9.dropna(axis = 0)

sinasc['ESTCIVMAE'] = df_cod_9['ESTCIVMAE']
sinasc['CONSULTAS'] = df_cod_9['CONSULTAS']


df_cod_9.isnull().sum()



ESTCIVMAE    0
CONSULTAS    0
dtype: int64

**Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero**

In [7]:
# 6) Seu código aqui
print(f"Linhas NAN: {df['QTDFILVIVO'].isnull().sum()}")
df['QTDFILVIVO'] = df['QTDFILVIVO'].replace(np.NaN, 0)
print(f"Linhas NAN apos tratadas: {df['QTDFILVIVO'].isnull().sum()}")

sinasc['QTDFILVIVO'] = df['QTDFILVIVO']

df

Linhas NAN: 1573
Linhas NAN apos tratadas: 0


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


**Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.**

In [8]:
# 7) seu código aqui
#df_grv = sinasc['GRAVIDEZ']
sinasc[sinasc['GRAVIDEZ'] == 'Ignorado'] = np.NaN



sinasc['GRAVIDEZ'].dropna(axis = 0, inplace = True)

print(sinasc['GRAVIDEZ'].isnull().sum())


79


In [9]:
print(sinasc['PARTO'].isnull().sum())

sinasc[sinasc['PARTO'] == 'Ignorado'] = np.NaN
print(sinasc['PARTO'].isnull().sum())

49
49


O Apgar possui uma classificação indicando se o bebê passou por asfixia:

Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'.
Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.

In [10]:
# 8) seu código aqui
print(sinasc['APGAR5'].value_counts())

sinasc.loc[sinasc['APGAR5'] > 7, ['APGAR_CATEGORIA']] = 'normal'
sinasc.loc[(sinasc['APGAR5'] > 5 )& (sinasc['APGAR5'] <= 7), ['APGAR_CATEGORIA']] = 'asfixia leve'
sinasc.loc[(sinasc['APGAR5'] > 3) & (sinasc['APGAR5'] <= 5), ['APGAR_CATEGORIA']] = 'asfixia moderada'
sinasc.loc[sinasc['APGAR5'] <= 3, ['APGAR_CATEGORIA']] = 'asfixia severa'



#sinasc['APGAR5'].tail(50)

9.0     16076
10.0     8393
8.0      1994
7.0       241
6.0        79
5.0        45
1.0        24
4.0        23
2.0        18
0.0        17
3.0        15
Name: APGAR5, dtype: int64


In [11]:
print(sinasc['APGAR_CATEGORIA'].value_counts())


normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: APGAR_CATEGORIA, dtype: int64


 Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True

In [30]:
# 9) seu código aqui

'''lista_1 = list(sinasc.columns)
print(lista_1)
'''
lista = ['ORIGEM', 'COD_ESTAB', 'COD_MUN_NASC', 
          'LOC_NASC', 'IDADE_MAE', 'EST_CIV_MAE', 
          'ESC_MAE', 'COD_OCUP_MAE', 'QTD_FIL_VIVO', 
          'QTD_FIL_MORT', 'COD_MUN_RES', 'GESTACAO', 
          'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DT_NASC', 
          'HORA_NASC', 'SEXO', 'APGAR_1', 'APGAR_5', 
          'RACA_COR', 'PESO', 'ID_ANOMAL', 'DT_CADASTRO', 
          'COD_ANOMAL', 'NUMERO_LOTE', 'VERSAO_SIST', 
          'DT_RECEBIM', 'DIF_DATA', 'DT_RECORIGA', 'NATURAL_MAE', 
          'COD_MUN_NATU', 'COD_UF_NATU', 'ESC_MAE_2010', 'SERIE_SC_MAE', 
          'DT_NASC_MAE', 'RACA_COR_MAE', 'QTD_GESTANT', 'QTD_PART_NOR', 
          'QTDPARTCES', 'IDADE_PAI', 'DT_ULT_MENST', 'SEMA_GESTAC', 
          'TP_MET_ESTIM', 'CONS_PRENAT', 'MES_PRENAT', 'TPA_PRESENT', 
          'ST_TRAB_PART', 'STCESPARTO', 'TP_NASC_ASSI', 'TP_FUNC_RESP', 
          'TP_DO_CRESP', 'DT_DECLARAC', 'ESC_MAE_AGR1', 'STD_NEPIDEM', 
          'ST_DN_NOVA', 'COD_PAIS_RES', 'TP_ROBSON', 'PARIDADE', 'KOTELCHUCK', 
          'CONTADOR', 'mun_Res_Status', 'mun_Res_Tipo', 'mun_Res_Nome', 'mun_Res_Uf', 
          'mun_Res_Lat', 'mun_Res_Lon', 'mun_Res_Alt', 'mun_Res_Area', 'APGAR_CATEGORIA']

In [32]:
lista_lower = [i.lower() for i in lista]
print(lista_lower)
df_snake = pd.DataFrame(sinasc)
df_snake.columns = lista_lower
df_snake


['origem', 'cod_estab', 'cod_mun_nasc', 'loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'cod_ocup_mae', 'qtd_fil_vivo', 'qtd_fil_mort', 'cod_mun_res', 'gestacao', 'gravidez', 'parto', 'consultas', 'dt_nasc', 'hora_nasc', 'sexo', 'apgar_1', 'apgar_5', 'raca_cor', 'peso', 'id_anomal', 'dt_cadastro', 'cod_anomal', 'numero_lote', 'versao_sist', 'dt_recebim', 'dif_data', 'dt_recoriga', 'natural_mae', 'cod_mun_natu', 'cod_uf_natu', 'esc_mae_2010', 'serie_sc_mae', 'dt_nasc_mae', 'raca_cor_mae', 'qtd_gestant', 'qtd_part_nor', 'qtdpartces', 'idade_pai', 'dt_ult_menst', 'sema_gestac', 'tp_met_estim', 'cons_prenat', 'mes_prenat', 'tpa_present', 'st_trab_part', 'stcesparto', 'tp_nasc_assi', 'tp_func_resp', 'tp_do_cresp', 'dt_declarac', 'esc_mae_agr1', 'std_nepidem', 'st_dn_nova', 'cod_pais_res', 'tp_robson', 'paridade', 'kotelchuck', 'contador', 'mun_res_status', 'mun_res_tipo', 'mun_res_nome', 'mun_res_uf', 'mun_res_lat', 'mun_res_lon', 'mun_res_alt', 'mun_res_area', 'apgar_categoria']


,origem,cod_estab,cod_mun_nasc,loc_nasc,idade_mae,est_civ_mae,esc_mae,cod_ocup_mae,qtd_fil_vivo,qtd_fil_mort,...,contador,mun_res_status,mun_res_tipo,mun_res_nome,mun_res_uf,mun_res_lat,mun_res_lon,mun_res_alt,mun_res_area,apgar_categoria
0,1.0,2679477.0,110001.0,1.0,19.0,5.0,8 a 11 anos,NaN,0.0,0.0,...,1.0,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal
1,1.0,2679477.0,110001.0,1.0,29.0,2.0,8 a 11 anos,999992.0,1.0,0.0,...,2.0,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal
2,1.0,2679477.0,110001.0,1.0,37.0,NaN,8 a 11 anos,513205.0,2.0,0.0,...,3.0,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal
3,1.0,2516500.0,110001.0,1.0,30.0,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4.0,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273,normal
4,1.0,2516500.0,110001.0,1.0,30.0,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5.0,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,1.0,2604388.0,510340.0,1.0,32.0,2.0,12 anos ou mais,333115.0,1.0,0.0,...,2767632.0,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915,normal
27024,1.0,2752654.0,510675.0,1.0,19.0,1.0,8 a 11 anos,999992.0,0.0,0.0,...,2772678.0,ATIVO,MUNIC,Chupinguaia,Rondônia,-12.55684,-60.90185,340.0,5126.723,normal
27025,1.0,9659366.0,510787.0,1.0,24.0,2.0,8 a 11 anos,421125.0,0.0,1.0,...,2776905.0,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915,normal
27026,1.0,9659366.0,510787.0,1.0,21.0,2.0,8 a 11 anos,999992.0,1.0,0.0,...,2777105.0,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915,normal
